<div class="notebook-buttons" style="display:flex; padding-top: 5rem;padding-bottom: 2.5rem;line-height: 2.15;">
    <a href="https://colab.research.google.com/github/zazuko/notebooks/blob/master/notebooks/electricity_prices/electricity_prices.ipynb">
        <div id="colab-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; align-items: center;">
            <img class="call-to-action-img" src="img/colab.svg" width="30" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Run in Google Colab</div>
        </div>
    </a>
    <a href="https://raw.githubusercontent.com/zazuko/notebooks/master/notebooks/electricity_prices/electricity_prices.ipynb" download>
        <div id="download-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/download.svg" width="22" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Download Notebook</div>
        </div>
    </a>
    <a href="https://github.com/zazuko/notebooks/blob/master/notebooks/electricity_prices/electricity_prices.ipynb">
        <div id="github-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/github.svg" width="25" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">View on GitHub</div>
        </div>
    </a>
</div>

# Swiss commerce register

Federal Office of Justice maintains ZEFIX, swiss commerce register. All legally operating businesses are in ZEFIX.


The register will provide us with company name, type, description, and address.   
ZEFIX is also available as [Linked Data](https://en.wikipedia.org/wiki/Linked_data). 


## Setup

### SPARQL endpoints

#### For companies data
Swiss commerce register is published as Linked Data. It can be accessed with [SPARQL queries](https://www.w3.org/TR/rdf-sparql-query/).   
You can send queries using HTTP requests. The API endpoint is **[https://lindas.admin.ch/query/](https://int.lindas.admin.ch/query).**  

#### For geodata
Different municipalities may have different tariffs. To understand their location, we will work with 
Swiss geodata. It is published as Linked Data. It can be accessed using API endpoint under **[https://ld.geo.admin.ch/query](https://ld.geo.admin.ch/query).**  

### SPARQL client

Let's use `SparqlClient` from [graphly](https://github.com/zazuko/graphly) to communicate with both databases. 
Graphly will allow us to:
* send SPARQL queries
* automatically add prefixes to all queries
* format response to `pandas` or `geopandas`

In [ ]:
import json
import re
import string

import folium
import mapclassify
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from graphly.api_client import SparqlClient

%matplotlib inline

In [ ]:
# Uncomment to install dependencies in Colab environment
#!pip install mapclassify
#!pip install git+https://github.com/zazuko/graphly.git

In [ ]:
sparql = SparqlClient("https://int.lindas.admin.ch/query")
geosparql = SparqlClient("https://ld.geo.admin.ch/query")

sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "admin": "<https://schema.ld.admin.ch/>"
})

geosparql.add_prefixes({
    "dct": "<http://purl.org/dc/terms/>",
    "geonames": "<http://www.geonames.org/ontology#>",
    "schema": "<http://schema.org/>",
    "geosparql": "<http://www.opengis.net/ont/geosparql#>",
})

SPARQL queries can become very long. To improve the readibility, we will work wih [prefixes](https://en.wikibooks.org/wiki/SPARQL/Prefixes).
 
Using the `add_prefixes` method, we can define persistent prefixes. 
Every time you send a query, `graphly` will now automatically add the prefixes for you.

# IDEAS
some zefix ideas for a notebook or something like it:

* show the address (street + number) with the most company registrations
* show the street with most company registrations
* show the municipality with most company registrations
* show them in relation to inhabitants of the municipality

# Companies by company type

In [ ]:
query = """
SELECT ?type (COUNT(DISTINCT ?company_iri) AS ?ccount)
WHERE {
    ?company_iri a admin:ZefixOrganisation.
    ?company_iri schema:additionalType/schema:name ?type .
  
  FILTER(LANG(?type) = "de")
}
GROUP BY ?type
ORDER BY ?ccount
"""

df = sparql.send_query(query)
df

In [ ]:
# Let's rename variables to english

de2en = {'Kommanditgesellschaft': "Limited Partnership",
         'Ausländische Niederlassung im Handelsregister eingetragen': "Foreign Branch",
         'Genossenschaft': "Cooperative",
         'Verein': "Association",
         'Kollektivgesellschaft': "General Partnership",
         'Schweizerische Zweigniederlassung im Handelsregister eingetragen': "Swiss Branch",
         'Stiftung': "Foundation",
         'Einzelunternehmen': "Sole proprietorship",
         'Aktiengesellschaft': "Joint-stock Company",
         'Gesellschaft mit beschränkter Haftung GMBH / SARL': "Limited Liability Company"}

df["type"] = df["type"].apply(lambda x: de2en[x] if x in de2en else x)


In [ ]:
fig = px.bar(df[df.ccount > 500], y="type", x="ccount", orientation = "h", labels={"type": "", "ccount": "companies count"})
fig.update_layout(
    title='Which company type is most popular?', 
    title_x=0.5,
)
fig.update_layout(bargap=0.40)
fig.show()

# Companies by municipality

In [ ]:
# Companies count
query = """
SELECT ?municipality ?muni_id (COUNT(?sub) AS ?companies)
FROM <https://lindas.admin.ch/foj/zefix>
FROM <https://lindas.admin.ch/territorial>
WHERE {
	?sub a admin:ZefixOrganisation ;
      admin:municipality ?muni_id.
    ?muni_id schema:name ?municipality;
} 
GROUP BY ?municipality ?muni_id
ORDER BY DESC(?companies)
"""

df = sparql.send_query(query)
df.head()

In [ ]:
# Communes
query = """    
SELECT ?muni_id ?population ?boundary 

WHERE {
  ?muni_iri dct:hasVersion ?version ;
            geonames:featureCode geonames:A.ADM3 .
  
  ?version schema:validUntil "2020-12-31"^^<http://www.w3.org/2001/XMLSchema#date>;
           geonames:population ?population;
           geosparql:hasGeometry/geosparql:asWKT ?boundary.
  
  BIND(IRI(REPLACE(STR(?muni_iri), "https://ld.geo.admin.ch/boundaries/", "https://ld.admin.ch/")) AS ?muni_id)
}

"""
communes = geosparql.send_query(query)
communes = communes.set_crs(epsg=4326)
communes["center"] = communes.to_crs(epsg=3035).centroid.to_crs(epsg=4326)

In [ ]:
join = pd.merge(communes, df, how="inner", on="muni_id")
join.sort_values(by="companies", ascending=False, inplace=True)
join[["municipality", "companies"]][0:10]

In [ ]:
N_CATEGORIES = 6
plot_df = join[join.companies > 2000].copy()

plot_df.loc[:,"text"] = plot_df.municipality + "<br>Companies registered: " + plot_df.companies.astype(int).astype(str)
classifier = mapclassify.NaturalBreaks(y=plot_df["companies"], k=N_CATEGORIES)
plot_df.loc[:, "buckets"] = plot_df[["companies"]].apply(classifier) 

norm = mpl.colors.Normalize(vmin=0, vmax=N_CATEGORIES)
colormap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.viridis)
labels = mapclassify.classifiers._get_mpl_labels(classifier, fmt="{:.0f}")

fig = go.Figure()

for bucket in range(N_CATEGORIES):

    subset = plot_df[plot_df.buckets == bucket]
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lat=subset.center.geometry.y,
        lon=subset.center.geometry.x,
        hovertext = subset.text,
        hoverinfo = "text",
        name=labels[bucket],
        marker={'size': ((subset.companies)*0.1)**1.03, "sizemode": "area", "sizemin": 4, "color": "rgba{}".format(colormap.to_rgba(bucket+1))}, 
    ))

fig.update_layout(
    margin={'l': 0, 't': 50, 'b': 0, 'r': 0},
    mapbox={
        'center': {"lat": 46.80515, "lon": 8.1336},
        'style': "carto-darkmatter",
        'zoom': 6.9},
    showlegend=True,
    legend_title="Registered companies",
    title_text='Where are most companies registered?', 
    title_x=0.5,
    width=980,
    height=600
)

fig.show("notebook")

In [ ]:
join["companies_per_inhabitants"] = round(join.companies/join.population*10000)
join["companies_per_inhabitants"] = join["companies_per_inhabitants"].astype(int)
join.sort_values(by="companies_per_inhabitants", ascending=False, inplace=True)
join[["municipality", "companies_per_inhabitants"]].head()

In [ ]:
N_CATEGORIES = 6
plot_df = join[join.companies_per_inhabitants > 1500].copy()

plot_df.loc[:,"text"] = plot_df.municipality + "<br>Companies: " + plot_df.companies_per_inhabitants.astype(int).astype(str)
classifier = mapclassify.NaturalBreaks(y=plot_df["companies_per_inhabitants"], k=N_CATEGORIES)
plot_df.loc[:, "buckets"] = plot_df[["companies_per_inhabitants"]].apply(classifier) 

norm = mpl.colors.Normalize(vmin=0, vmax=N_CATEGORIES)
colormap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.viridis)
labels = mapclassify.classifiers._get_mpl_labels(classifier, fmt="{:.0f}")

fig = go.Figure()

for bucket in range(N_CATEGORIES):

    subset = plot_df[plot_df.buckets == bucket]
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lat=subset.center.geometry.y,
        lon=subset.center.geometry.x,
        hovertext = subset.text,
        hoverinfo = "text",
        name=labels[bucket],
        marker={'size': ((subset.companies_per_inhabitants)*0.03)**1.55, "sizemode": "area", "sizemin": 4, "color": "rgba{}".format(colormap.to_rgba(bucket+1))}, 
    ))

fig.update_layout(
    margin={'l': 0, 't': 50, 'b': 0, 'r': 0},
    mapbox={
        'center': {"lat": 46.80515, "lon": 8.1336},
        'style': "carto-darkmatter",
        'zoom': 6.9},
    showlegend=True,
    legend_title="Companies per<br>10 000 inhabitants",
    title_text='Where are most companies registered?', 
    title_x=0.5,
    width=980,
    height=600
)

fig.show("notebook")

In [ ]:
# Same plot as heatmap

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

highlight_function = lambda x: {'fillColor': '#989898', 
                                'color':'#000000', 
                                'fillOpacity': 0.8}

def plot_heatmap(df, variable, variable_description, title):
    
    classifier = mapclassify.NaturalBreaks(y=df[variable], k=5)
    bins = [df[variable].min()] + list(classifier.bins)

    m = folium.Map(location=[46.83, 8.13], zoom_start=8, tiles="cartodbpositron")

    folium.Choropleth(
        geo_data=json.loads(df.to_json()),
        data=df,
        columns=["municipality", variable],
        key_on="feature.properties.municipality",
        fill_color="YlOrRd",
        fill_opacity=1,
        line_weight=0,
        smooth_factor=0,
        bins=bins,
        reset=True,
        legend_name=variable_description,
    ).add_to(m)

    hover = folium.features.GeoJson(
        df,
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipality', variable],
            aliases=['Municipality: ', variable_description + ": "],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )

    folium.LayerControl().add_to(m)
    m.add_child(hover)
    m.keep_in_front(hover)
    
    title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(title)   
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [ ]:
join = join.drop(columns="center")
plot_heatmap(join, "companies_per_inhabitants", "Companies per 10.000 inhabitants", "Where are most companies registered?")